## **Zadanie 2 - Algorytm genetyczny**

Cel zadania polega na implementacji algorytmu genetycznego z mutacją, selekcją ruletkową, krzyżowaniem jednopunktowym oraz sukcesją generacyjną.

Zaimplementowany algorytm ma następnie posłużyć do optymalizacji klasycznego, symetrycznego [problemu komiwojażera](https://pl.wikipedia.org/wiki/Problem_komiwojażera).

**Dane:**
| Miasto | Współrzędne |
|--------|-------------|
| A      | (0, 0)      |
| B      | (1, 3)      |
| C      | (2, 1)      |
| D      | (4, 6)      |
| E      | (5, 2)      |
| F      | (6, 5)      |
| G      | (8, 7)      |
| H      | (9, 4)      |
| I      | (10, 8)     |
| J      | (12, 3)     |

**Kroki do wykonania**
- Implementacja algorytmu genetycznego.
- Zastosowanie algorytmu do rozwiązania problemu komiwojażera. Eksperymentalne dobranie zestawu parametrów, dla którego algorytm daje dobry wynik.
- Zbadanie, w jaki sposób następujące zmiany wpłyną na rezultaty osiągane przez algorytm:
  - Zwiększenie prawdopodobieństwa mutacji.
  - Zmiana sposobu selekcji na turniejową.

**Uwagi**
- Ze względu na losowy charakter algorytmu, konieczne jest porównanie uzyskiwanych rezultatów jego działania dla wielu uruchomień i uśrednienie wyników.
- Funkcję przystosowania możemy zdefiniować jako odwrotność łącznej długości trasy dla danej permutacji miast odwiedzanych przez komiwojażera.
- Interesującym, choć nieobowiązkowym dodatkiem byłoby zastosowanie do kodowania osobników kodu Graya. Szczegóły omówione są w podrozdziale 4.2 książki Pawła Wawrzyńskiego Podstawy sztucznej inteligencji (dostępna m.in. online).

In [ ]:
import copy
from typing import List, Tuple, Optional, Callable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

### Implementacja algorytmu genetycznego

In [ ]:
GENES = [0, 1]
POPULATION_SIZE = 100
MUTATION_PROBABILITY = 0.01
CROSSOVER_PROBABILITY = 0.7
MAX_GENERATIONS = 100

**Osobonik** reprezentuje punkt w przeszukiwanej przestrzeni. W przypadku problemu komiwojażera jest to permutacja miast.

In [ ]:
class Individual:
    def __init__(self, ind_length: int = None, genes: Optional[np.ndarray] = None) -> None:
        if ind_length is None and genes is None:
            raise ValueError('Either genes or ind_length must be provided')
        self.genes = genes if genes is not None else self.generate_individual(ind_length)

    def __str__(self) -> str:
        return f'{self.genes}'

    def __len__(self) -> int:
        return len(self.genes)

    def generate_individual(self, ind_length: int) -> np.ndarray:
        return np.random.randint(2, size=ind_length)

In [ ]:
ind = Individual(10)
print(f"Generated individual: {ind} with length {len(ind)}.")

**Populacja** to zbiór osobników przetwarzanych w każdej iteracji algorytmu.

In [ ]:
class Population:
    def __init__(self, individuals: Optional[List[Individual]]) -> None:
        self.individuals = individuals

    def __str__(self):
        for i, ind in enumerate(self.individuals):
            print(f'Individual {i + 1}: {ind}')
        return ''

    def __len__(self):
        return len(self.individuals)

    def get_fitness_values(self, objective_function: Callable[[np.ndarray], float]) -> List[float]:
        return [objective_function(ind.genes) for ind in self.individuals]

    def get_best_individual(self, fitness_values: List[float]) -> Tuple[Individual, float]:
        return self.individuals[np.argmin(fitness_values)], np.min(fitness_values)


In [ ]:
individuals = [Individual(10) for _ in range(POPULATION_SIZE)]
population = Population(individuals)
print(f"Generated population with {len(population)} individuals.")

In [ ]:
def objective_function(ind_genes: np.ndarray) -> float:
    return -np.sum(ind_genes)

In [ ]:
population_fitness = population.get_fitness_values(objective_function)
best_individual, best_fitness = population.get_best_individual(population_fitness)
print(f"Best individual: {best_individual} with fitness {best_fitness}.")

In [ ]:
class GeneticAlgorithmPath:
    def __init__(
            self,
            objective_function: Callable[[np.ndarray], float],
            initial_population: Population,
            population_size: int,
            mutation_prob: float,
            crossover_prob: float,
            max_generations: int
        ) -> None:
        self.objective_function = objective_function
        self.population = initial_population
        self.population_size = population_size
        self.mutation_prob = mutation_prob
        self.crossover_prob = crossover_prob
        self.max_generations = max_generations

    def run(self) -> Tuple[Individual, float]:
        t = 0
        o = self.population.get_fitness_values(self.objective_function)
        x_best, o_best = self.population.get_best_individual(o)

        while t < self.max_generations:
            R = self.roulette_wheel_selection(self.population, o, self.population_size)
            M = self.crossover_and_mutation(R)
            o = M.get_fitness_values(self.objective_function)

            x_best_new, o_best_new = M.get_best_individual(o)

            if o_best_new <= o_best:
                x_best, o_best = x_best_new, o_best_new

            self.population = M
            t += 1

        print(f'Generation {t}: best individual {x_best} with fitness {o_best}')

        return x_best, o_best


    def roulette_wheel_selection(self, population_t, fitness_values, population_size):
        pop_individuals = population_t.individuals
        probabilities = fitness_values / np.sum(fitness_values)
        return Population(np.random.choice(pop_individuals, size=population_size, p=probabilities))

    def crossover_and_mutation(self, R):
        new_population = []

        individuals_t = R.individuals

        for i in range(0, len(individuals_t), 2):
            parent1, parent2 = individuals_t[i], individuals_t[i + 1]
            child1, child2 = None, None

            if np.random.rand() < self.crossover_prob:
                crossover_point = np.random.randint(0, len(parent1))
                child1 = np.concatenate((parent1.genes[:crossover_point], parent2.genes[crossover_point:]))
                child2 = np.concatenate((parent2.genes[:crossover_point], parent1.genes[crossover_point:]))

            if child1 is not None and child2 is not None:
                new_population.append(Individual(genes=child1))
                new_population.append(Individual(genes=child2))
            else:
                new_population.append(parent1)
                new_population.append(parent2)


        for i in range(len(new_population)):
            for j in range(len(new_population[i])):
                if np.random.rand() < self.mutation_prob:
                    new_population[i].genes[j] = 1 - new_population[i].genes[j]

        return Population(new_population)


In [ ]:
population = Population([Individual(100) for _ in range(POPULATION_SIZE)])

In [ ]:
ga = GeneticAlgorithmPath(objective_function, population, POPULATION_SIZE, MUTATION_PROBABILITY, CROSSOVER_PROBABILITY, MAX_GENERATIONS)

x_best, o_best = ga.run()

### Zastosowanie algorytmu do rozwiązania problemu komiwojażera

In [ ]:
city_data = {
    'Miasto': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'x': [0, 1, 2, 4, 5, 6, 8, 9, 10, 12],
    'y': [0, 3, 1, 6, 2, 5, 7, 4, 8, 3]
}

In [ ]:
df = pd.DataFrame(city_data)
df.head()

In [ ]:
def visualize_map(df, paths=None):
    plt.figure(figsize=(8, 8))
    plt.scatter(df['x'], df['y'], s=200)

    for _, row in df.iterrows():
        plt.text(row['x'], row['y'], row['Miasto'], fontsize=10, ha='center', va='center')

    if paths:
        for path in paths:
            x_values = [df.loc[i, 'x'] for i in path]
            y_values = [df.loc[i, 'y'] for i in path]
            plt.plot(x_values, y_values, marker='o')

    plt.title('Visualization of Cities')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.show()

In [ ]:
cities_list = df['Miasto'].values
cities_list

In [ ]:
np.random.permutation(cities_list)

In [ ]:
class IndividualPath:
    def __init__(self, genes: Optional[np.ndarray] = None, cities: Optional[np.ndarray] = None) -> None:
        if genes is None and cities is None:
            raise ValueError('Either genes or cities must be provided')
        self.genes = genes if genes is not None else self.generate_individual(cities)

    def __str__(self):
        return f'{self.genes}'

    def __len__(self):
        return len(self.genes)

    def generate_individual(self, cities: np.ndarray) -> np.ndarray:
        return np.random.permutation(cities)

In [ ]:
ind = IndividualPath(cities=cities_list)
print(f"Generated individual: {ind} with length {len(ind)}.")

In [ ]:
def calculate_distance(city1, city2, df):
    x1, y1 = df[df['Miasto'] == city1].values[0][1:]
    x2, y2 = df[df['Miasto'] == city2].values[0][1:]

    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def objective_function_distance(ind_genes: np.ndarray, df: pd.DataFrame = df) -> float:
    return 1 / (sum([calculate_distance(ind_genes[i], ind_genes[i + 1], df) for i in range(len(ind_genes) - 1)]) + calculate_distance(ind_genes[-1], ind_genes[0], df))

In [ ]:
objective_function_distance(ind.genes, df)

In [ ]:
class PopulationPath:
    def __init__(self, individuals: List[IndividualPath]) -> None:
        self.individuals = individuals

    def __str__(self):
        for i, ind in enumerate(self.individuals):
            print(f'Individual {i + 1}: {ind}')
        return ''

    def __len__(self):
        return len(self.individuals)

    def get_fitness_values(self, objective_function: Callable[[np.ndarray], float]) -> List[float]:
        return [objective_function(ind.genes) for ind in self.individuals]

    def get_best_individual(self, fitness_values: List[float]) -> Tuple[IndividualPath, float]:
        return self.individuals[np.argmax(fitness_values)], np.max(fitness_values)

In [ ]:
individuals = [IndividualPath(cities=cities_list) for _ in range(200)]
population = PopulationPath(individuals)

Funkcja przystosowania dla problemu komiwojażera to odwrotność łącznej długości trasy dla danej permutacji miast odwiedzanych przez komiwojażera.

In [ ]:
class GeneticAlgorithmPath:
    def __init__(
            self,
            objective_function: Callable[[np.ndarray], float],
            initial_population: PopulationPath,
            population_size: int,
            mutation_prob: float,
            crossover_prob: float,
            max_generations: int,
            selection_method: str ='roulette_wheel',
            print_results: bool = False
        ):
        self.objective_function = objective_function
        self.population = initial_population
        self.population_size = population_size
        self.mutation_prob = mutation_prob
        self.crossover_prob = crossover_prob
        self.max_generations = max_generations
        self.selection_method = selection_method
        self.print_results = print_results

    def run(self) -> Tuple[IndividualPath, float]:
        o = self.population.get_fitness_values(self.objective_function)
        x_best, o_best = self.population.get_best_individual(o)

        for t in tqdm(range(self.max_generations)):
            if self.print_results and t % 10 == 0:
                print(f'Generation {t}: best individual {x_best} with fitness {o_best}')
            if self.selection_method == 'roulette_wheel':
                R = self.roulette_wheel_selection(self.population, o)
            elif self.selection_method == 'tournament':
                R = self.tournament_selection(self.population, o)
            M = self.crossover_and_mutation(R)
            o = M.get_fitness_values(self.objective_function)
            x_best_new, o_best_new = M.get_best_individual(o)

            if o_best_new > o_best:
                x_best, o_best = x_best_new, o_best_new

            self.population = M

        return x_best, o_best


    def roulette_wheel_selection(self, population_t: PopulationPath, fitness_values: List[float]) -> PopulationPath:
        pop_individuals = population_t.individuals
        probabilities = fitness_values / np.sum(fitness_values)
        return PopulationPath(np.random.choice(pop_individuals, size=self.population_size, p=probabilities))

    def tournament_selection(self, population_t: PopulationPath, fitness_values: List[float]) -> PopulationPath:
        k = 2
        pop_individuals = population_t.individuals
        selected = []
        for _ in range(self.population_size):
            idx = np.random.choice(range(len(population_t)), k)
            if fitness_values[idx[0]] > fitness_values[idx[1]]:
                selected.append(pop_individuals[idx[0]])
            else:
                selected.append(pop_individuals[idx[1]])

        return PopulationPath(selected)

    def crossover_and_mutation(self, R: PopulationPath) -> PopulationPath:
        new_population = []

        current_population = copy.copy(R.individuals)
        for i in range(0, len(current_population), 2):
            parent1, parent2 = current_population[i], current_population[i + 1]
            child1, child2 = None, None

            if np.random.rand() < self.crossover_prob:
                crossover_point = np.random.randint(0, len(parent1))
                child1 = parent1.genes[:crossover_point]
                child2 = parent2.genes[:crossover_point]

                for city in parent2.genes:
                    if city not in child1:
                        child1 = np.append(child1, city)

                for city in parent1.genes:
                    if city not in child2:
                        child2 = np.append(child2, city)

            if child1 is not None and child2 is not None:
                new_population.append(IndividualPath(child1))
                new_population.append(IndividualPath(child2))
            else:
                new_population.append(parent1)
                new_population.append(parent2)

        for i in range(len(new_population)):
            for j in range(len(new_population[i])):
                if np.random.rand() < self.mutation_prob:
                    idx1, idx2 = np.random.choice(len(new_population[i]), 2)
                    new_population[i].genes[idx1], new_population[i].genes[idx2] = new_population[i].genes[idx2], new_population[i].genes[idx1]

        return PopulationPath(new_population)

In [ ]:
population = PopulationPath([IndividualPath(cities=cities_list) for _ in range(300)])

In [ ]:
population_t = PopulationPath([IndividualPath(cities=cities_list) for _ in range(200)])
ga_city = GeneticAlgorithmPath(objective_function_distance, population_t, len(population_t), 0.01, 0.8, 200)

best_path, best_fitness = ga_city.run()

In [ ]:
collect_best = []

for _ in range(10):
    population_t = copy.deepcopy(population)
    ga_city = GeneticAlgorithmPath(objective_function_distance, population_t, len(population_t), 0.01, 0.8, 200)
    x_best, o_best = ga_city.run()
    print(f'Best individual: {x_best} with fitness {o_best}')
    collect_best.append((x_best, o_best))

In [ ]:
def visualize_map_with_path(path, df):
    plt.figure(figsize=(6,6))
    plt.scatter(df['x'], df['y'], s=200)

    for _, row in df.iterrows():
        plt.text(row['x'], row['y'], row['Miasto'], fontsize=10, ha='center', va='center')

    for i in range(len(path) - 1):
        city1 = path[i]
        city2 = path[i + 1]

        x1, y1 = df[df['Miasto'] == city1].values[0][1:]
        x2, y2 = df[df['Miasto'] == city2].values[0][1:]

        plt.plot([x1, x2], [y1, y2], 'k-')

    plt.plot([df[df['Miasto'] == path[-1]].values[0][1], df[df['Miasto'] == path[0]].values[0][1]], [df[df['Miasto'] == path[-1]].values[0][2], df[df['Miasto'] == path[0]].values[0][2]], 'k-')

    plt.title(f'Wizualizacja ścieżki {"".join(path)}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

In [ ]:
visualize_map_with_path(best_path.genes, df)

In [ ]:
def calculate_path_total_distance(path, df):
    return sum([calculate_distance(path[i], path[i + 1], df) for i in range(len(path) - 1)]) + calculate_distance(path[-1], path[0], df)

In [ ]:
calculate_path_total_distance(best_path.genes, df)

### Zbadanie wpływu zmian na rezultaty osiągane przez algorytm

#### Zwiększenie prawdopodobieństwa mutacji

In [ ]:
x_best_path, o_best_path, probs = [], [], []
prob_mutation = [0.05, 0.1, 0.2, 0.3, 0.4]

for prob in prob_mutation:
    for _ in range(5):
        population = copy.deepcopy(population)
        ga_city = GeneticAlgorithmPath(objective_function_distance, population, len(population), prob, 0.8, 200, selection_method='tournament', print_results=True)
        x_best, o_best = ga_city.run()
        x_best_path.append(x_best)
        o_best_path.append(o_best)
        probs.append(prob)

In [ ]:
for i, path in enumerate(x_best_path):
    print(f'Path {i + 1}: {path} with total distance {calculate_path_total_distance(path.genes, df)}')

In [ ]:
results = pd.DataFrame({
    'Mutation Probability': probs,
    'Path': x_best_path,
    'Total Score': o_best_path
})

results.head()

In [ ]:
# create a new df with mean, std and count of the results for each mutation probability. Add min and max values for the total distance
results_grouped = results.groupby('Mutation Probability').agg(
    Mean_Total_Distance=('Total Score', 'mean'),
    Std_Total_Distance=('Total Score', 'std'),
    Count=('Total Score', 'count'),
    Min_Total_Distance=('Total Score', 'min'),
    Max_Total_Distance=('Total Score', 'max')
)

results_grouped.head()

In [ ]:
# visualize the results

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)

plt.errorbar(results_grouped.index, results_grouped['Mean_Total_Distance'], yerr=results_grouped['Std_Total_Distance'], fmt='o', capsize=5)

plt.title('Średnia wartość odwronej odległości w zależności od prawdopodobieństwa mutacji')
plt.xlabel('Prawdopodobieństwo mutacji')
plt.ylabel('Średnia wartość odległości')
plt.grid(True)

#### Zmiana sposobu selekcji na turniejową

In [ ]:
tournament_results, roulette_wheel_results = [], []

for _ in range(5):
    population = copy.deepcopy(population)
    ga_city = GeneticAlgorithmPath(objective_function_distance, population, len(population), 0.001, 0.8, 200, selection_method='tournament')
    x_best, o_best = ga_city.run()
    tournament_results.append((x_best, o_best))

for _ in range(5):
    population = copy.deepcopy(population)
    ga_city = GeneticAlgorithmPath(objective_function_distance, population, len(population), 0.001, 0.8, 200)
    x_best, o_best = ga_city.run()
    roulette_wheel_results.append((x_best, o_best))

In [ ]:
for i, (path, score) in enumerate(tournament_results):
    print(f'Tournament Path {i + 1}: {path} with total distance {calculate_path_total_distance(path.genes, df)}')

for i, (path, score) in enumerate(roulette_wheel_results):
    print(f'Roulette Wheel Path {i + 1}: {path} with total distance {calculate_path_total_distance(path.genes, df)}')